# 이미지 처리 LeNet

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, AveragePooling2D, Flatten, Dense, ZeroPadding2D

from sklearn.model_selection import train_test_split

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)

x_train = x_train/255.0
x_val = x_val/255.0
x_test = x_test/255.0

In [ ]:
LeNet = Sequential([InputLayer(input_shape=(28,28,1)), 
                    ZeroPadding2D((2,2)),
                    Conv2D(6,5, activation="tanh"),
                    AveragePooling2D(strides=2), # 2x2 크기라서 strides=2, 그리고 Conv에서의 strides와 다름
                    Conv2D(16,5,activation="tanh"),
                    AveragePooling2D(strides=2),
                    Conv2D(120,5, activation="tanh"),
                    Flatten(), # FC layer 과정
                    Dense(84,activation="tanh"), # FC layer 과정
                    Dense(10,activation="softmax")])

In [ ]:
LeNet.compile(optimizer="SGD",
             loss="sparse_categorical_crossentropy",
             metrics="accuracy")

In [ ]:
LeNet.summary()

In [ ]:
LeNet.fit(x_train, y_train, epochs=10)

In [ ]:
LeNet.evaluate(x_test, y_test)

# AlexNet

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

In [ ]:
# 데이터 불러오기
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory="./dogs_vs_cats/train",
    labels="inferred",
    label_mode='int',
    batch_size=32,
    image_size=(227,227)
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    directory="./dogs_vs_cats/test",
    labels="inferred", # labels are generated from the directory structure
    label_mode='int', # 0=cats & 1=dogs
    batch_size=32,
    image_size=(227,227)
)

In [ ]:
# 데이터 정규화하기
# Normalize
def process(image, label):
    image = tf.cast(image/255., tf.float32)
    return image, label

train_ds = train_ds.map(process)
test_ds = test_ds.map(process)

In [ ]:
model = Sequential(name='alex_net')
model.add(Conv2D(filters=96, kernel_size=11,strides=4, activation='relu',input_shape=(227,227,3)))
model.add(MaxPool2D(pool_size=3,strides=2))

model.add(Conv2D(filters=256, kernel_size=5, activation='relu',padding="same"))
model.add(MaxPool2D(pool_size=3,strides=2))

model.add(Conv2D(filters=384, kernel_size=3,strides=1,activation='relu',padding="same"))
model.add(Conv2D(filters=384,kernel_size=3,strides=1,activation='relu',padding="same"))
model.add(Conv2D(filters=256,kernel_size=3,strides=1,activation='relu',padding="same"))
model.add(MaxPool2D(pool_size=3,strides=2))

model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5)) # 가중치 선택(drop), 정규화 기법 중 하나, 주로 Dense에서 많이 함
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1 ,activation='sigmoid')) # 가장 확률이 높은 것으로 클래스를 분류

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
# 모델 적용하기
model.fit(train_ds, epochs=10,validation_data=test_ds)

# VGGNet 
- 레이어의 갯수가 크면 더 성능이 좋지 않을까?를 실험한 모델

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

In [ ]:
# 모델 선언하기
vgg = Sequential(name="vgg16")
vgg.add(Conv2D(64,3, strides = 1, activation="relu", padding="same", input_shape=(227,227,3)))
vgg.add(Conv2D(64,3, strides = 1, activation="relu", padding="same")) # 3x3필터, 64개의 필터
vgg.add(MaxPool2D(2,2)) # strides = 2랑 (2,2)랑 같음
vgg.add(Conv2D(128,3, strides=1, activation="relu", padding="same"))
vgg.add(Conv2D(128,3, strides=1, activation="relu", padding="same"))
vgg.add(MaxPool2D(2,2))
vgg.add(Conv2D(256,3, strides=1, activation="relu", padding="same"))
vgg.add(Conv2D(256,3, strides=1, activation='relu', padding="same"))
vgg.add(Conv2D(256,1, strides=1, activation='relu', padding="same"))
vgg.add(MaxPool2D(2,2))
vgg.add(Conv2D(512,3, strides=1, activation="relu", padding="same"))
vgg.add(Conv2D(512,3, strides=1, activation='relu', padding="same"))
vgg.add(Conv2D(512,1, strides=1, activation='relu', padding="same"))
vgg.add(MaxPool2D(2,2))
vgg.add(Conv2D(512,3, strides=1, activation="relu", padding="same"))
vgg.add(Conv2D(512,3, strides=1, activation='relu', padding="same"))
vgg.add(Conv2D(512,1, strides=1, activation='relu', padding="same"))
vgg.add(MaxPool2D(2,2))
vgg.add(Flatten())
vgg.add(Dense(4096,activation='relu'))
vgg.add(Dense(4096,activation='relu'))
vgg.add(Dense(1,activation="sigmoid")) # 이진분류일때는 output 클래스 분류 갯수가 1개만 이어도 괜찮음
# 왜냐하면 하나가 1(True)이면 나머지는 자동으로 0(False)이니깐!
# class분류가 3개 이상이면 softmax, 분류가 2개면 sigmoid

In [ ]:
vgg.summary()

In [ ]:
vgg.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
# 분류가 3개 이상이면 sparse_categorical_crossentropy 이거나 categorical_crossentropy 사용!
# binary_crossentropy는 이진분류

In [ ]:
# 모델 적용하기
vgg.fit(train_ds, epochs=10,validation_data=test_ds)